In [ ]:
# If not installed, run this cell. It is needed to read DBF files
#!pip install dbfread
#!pip show dbfread

In [1]:
import arcpy
from arcpy import env
from arcpy.sa import *
from otherfunctions import folders_exist

In [2]:
# Paths to input datasets
root_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs"
tam_out_dir = r"Z:\PhD_Datasets&Analysis\Outputs\T&M_WBM"
tc_ds = root_folder + "\\TerraClimate"
out_geotiff = tc_ds + "\\GeoTIFF"
bands_gee = ["pr", "pet", "ro"] # band names in GEE - for comparison with GEE TerraClimate dataset
tc_vars = ["ppt", "pet", "q"] # variable names according to TerraClimate


# Set arcpy environment variables
env.overwriteOutput = True
arcpy.CheckOutExtension("spatial")
# env.cellSize = "MINOF" # Avoided to prevent huge files
env.cellSize = out_geotiff + "\\" + tc_vars[0] + "_2023_1.tif" # Use TerraClimate resolution as reference for cell size
env.workspace = r"Z:\PhD_Datasets&Analysis\_ProcessingCache"
env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984") # WGS 1984 (4326)

In [3]:
# Get the current environment's spatial reference
spatial_ref = env.outputCoordinateSystem

# Check if a spatial reference is set
if spatial_ref:
    print(f"Spatial Reference Name: {spatial_ref.name}")
    print(f"Spatial Reference WKID: {spatial_ref.factoryCode}")
else:
    print("No spatial reference is set in the current environment.")

Spatial Reference Name: GCS_WGS_1984
Spatial Reference WKID: 4326


In [ ]:
# Read the water holding capacity (WHC) raster
whc = Float(Raster(r"Z:\PhD_Datasets&Analysis\Inputs\WHC_FC33_Final_WGS84.tif")) / 1000 # As the raster was originally multiplied by 1000
whc = Con(whc == 0, 1/1000000, whc) # Avoid division by zero in subsequent calculations (sstor)

In [4]:
######################################################
### Starting values for the water balance model - T&M
######################################################

# Initial variables
years = range(1958, 2023 + 1) # Years with available weather information to run the water balance
months = range(1, 12 + 1)

# Base flow of the previous month (mm) (bf0)
bflow_ant = 10

In [6]:
# Initial soil water storage expressed as a fraction of water holding capacity [0-1]
ffcb = 0.1

# Initial soil water storage (st0)
sstor_ant = whc * ffcb

In [7]:
# Create folders for other variables of tam model
eprec_dir = tam_out_dir + '\\eprec'
sstor_dir = tam_out_dir + '\\sstor'
aet_dir = tam_out_dir + '\\aet'
perc_dir = tam_out_dir + '\\perc'

folders_exist([eprec_dir, sstor_dir, aet_dir, perc_dir])

In [ ]:
print('\n############################################################')
print('\t\tINITIAL VARIABLES')
print('\tPeriod to be executed: ' + str(years[0]) + '-' + str(years[-1]))
print('############################################################')

for year in years:

    print("\n**Executing THORNTHWAITE AND MATHER model for " + str(year) + "**")

    for month in months:

        print("\n\t*Executing water balance for month " + str(month) + "*\n")

        print("\tLoading precipitation, potential evapo., and runoff rasters from TerraClimate GeoTIFF files.....")

        pr_name = tc_vars[0] + "_" + str(year) + "_" + str(month)
        pet_name = tc_vars[1] + "_" + str(year) + "_" + str(month)
        ro_name = tc_vars[2] + "_" + str(year) + "_" + str(month)

        pr = Raster(out_geotiff + "\\" + pr_name + ".tif")
        pet = Raster(out_geotiff + "\\" + pet_name + ".tif")
        ro = Raster(out_geotiff + "\\" + ro_name + ".tif")

        print("\tPrecipitation, potential evapo., and runoff were loaded")

        print("\tCalculating effective precipitation......")
        eprec = Con(pr - ro > 0, pr - ro, 0) # Needs to be positive, otherwise it is 0
        eprec.save(eprec_dir + "\\eprec_" + str(year) + "_" + str(month) + ".tif")

        print("\tCalculating soil storage......")
        sstor = Con(eprec <= pet, sstor_ant * Exp(-Abs(eprec - pet) / whc), Con(sstor_ant + (eprec - pet) > whc, whc, sstor_ant + (eprec - pet)))
        sstor.save(sstor_dir + "\\sstor_" + str(year) + "_" + str(month) + ".tif")

        print("\tCalculating actual evapotranspiration......")
        aet = Con(eprec > pet, pet, Con(eprec + sstor - sstor_ant > 0, eprec + sstor - sstor_ant, 0))
        aet.save(aet_dir + "\\aet_" + str(year) + "_" + str(month) + ".tif")

        print("\tCalculating percolation......")
        perc = Con(eprec <= pet, 0, Con(sstor_ant + (eprec - pet) > whc, sstor_ant + (eprec - pet) - whc, 0))
        perc.save(perc_dir + "\\perc_" + str(year) + "_" + str(month) + ".tif")

        # sstor_ant is si-1 for other months different from the first one 
        sstor_ant = sstor

        # Delete variables to save memory
        print("\tDeleting variables......")
        del pr, pet, ro, eprec, sstor, aet, perc

arcpy.CheckInExtension("spatial")

# Clear the workspace environment
arcpy.ClearEnvironment("workspace")

print("\nDONE!!")